# 2.2 Station meta data handling in Pyrocko

This chapter will show multiple steps of station meta data handling in *Pyrocko* as
* StationXML loading
* Station extraction
* Station manipulation
* Pyrockos internal station format
* Control of the instrument response
* Instrument response removal of recorded waveforms.

## Contents
* [Loading and first display](#sec1)
* [Manually add a station](#sec2)
* [Saving](#sec3)
* [Instrument response and its removal](#sec4)
* [Summary](#sum)


## Loading and first display  <a class="anchor" id="sec1"></a>

*Pyrocko* loads station meta data via `stationxml.load_xml` or using the `Squirrel.

In [ ]:
# Import of the loading function
from pyrocko.io import stationxml
from pyrocko.squirrel import Squirrel

# Stations are loaded into Pyrocko StationXML 
sx = stationxml.load_xml(filename='data/stations.xml');

# OR into the Squirrel
sq = Squirrel()
sq.add(['data/stations.xml']);

For most cases *Pyrocko* works with its own data structure, in which stations are gather as a list of Pyrocko `Station`s. Conversion is done with `get_pyrocko_stations()`:

In [ ]:
# Convert stations into Pyrocko format and print the station name
stations = sx.get_pyrocko_stations()

for s in stations:
    print('{}.{:<4}: lat [deg] {:.4f}, lon [deg] {:.4f}'.format(s.network, s.station, s.lat, s.lon))
    
# OR one uses the Squirrel
for s in sq.get_stations():
    print('Squirrel: {}.{:<4}: lat [deg] {:.4f}, lon [deg] {:.4f}'.format(
        s.codes.network, s.codes.station, s.lat, s.lon))

Map plots allowing for an inspection of the station locations are easily done within `Snuffler`. Just start it with the stations and search for the snuffling `Map`:

In [ ]:
# Load Snuffler
from pyrocko.trace import snuffle

# Start snuffler with the stations
snuffle([], stations=stations)

# OR snuffle the squirrel
snuffle([], stations=sq.get_stations(model='pyrocko'))

## Manually add a station  <a class="anchor" id="sec2"></a>

Of course it is also possible to add stations manually, e.g. from your own campaign. So let's assume the hypothetical station `GE.RHD` installed on Rhodes (latitude: 36.415597 deg, longitude: 28.193360, elevation: 50 m). It has only a vertical component. We want to add this station to our station list assuming the network `GE`.
Finally we want to convert it into stationxml format.

First we need to import *Pyrockos*s `Station` and `Channel` class

In [ ]:
from pyrocko.model import Station, Channel

In the next step we will create a Station with the given location and code:

In [ ]:
station = Station(
    # Station code
    network='GE',
    station='RHD',
    location='',
    # Location
    lat=36.415597,
    lon=28.193360,
    elevation=50.)

Now we can add the channel information:

In [ ]:
channel = Channel(
    # Channel code
    name='HHZ')

# Add channel list to the station
station.add_channel(channel)

Finally we add the station to the list of stations and generate a stationxml object using `FDSNStationXML.from_pyrocko_stations`:

In [ ]:
# Add station to station list
stations.append(station)

# Generate stationxml object
sx_new = stationxml.FDSNStationXML.from_pyrocko_stations(stations)

When now printing the inventory stats, the freshly added station `GE.RHD` appears:

In [ ]:
print(sx_new)

## Saving  <a class="anchor" id="sec3"></a>

As we have added the new station, we should also save the new StationXML. The stationxml object allows to do so easily with the `dump_xml` method:

In [ ]:
sx_new.dump_xml(filename='processed_stations.xml')

Note: We can also save the stations in Pyrockos YAML-serialized format using `dump_stations`:

In [ ]:
# Import of station saving function
from pyrocko.model import dump_stations

# Writing of station data into YAML file
dump_stations([stations], filename='processed_stations_pyrocko.yaml')

## Instrument response and its removal  <a class="anchor" id="sec4"></a>

What can we use the station information for? Perhaps **restitution**, so the removal of the instrumental effects (its response) from the recorded raw waveforms. In our case let's try to remove the instrument response from the waveform record, we have seen in [1.2 Waveform handling with Pyrocko](1.2_waveforms_pyrocko.ipynb).

So, first load the waveform into the Squirrel (following the [example](https://pyrocko.org/docs/current/library/examples/squirrel/cli_tool.html)):

In [ ]:
# Choosing an individual trace (here the first) of the traces list extracted from squirrel
sq.add(['data/data_GE.KTHA..HHZ_2020-10-30_11-30-26.mseed'])

traces_squirrel = sq.get_waveforms(codes=[('GE', 'KTHA', '', 'HHZ')])
tr = traces_squirrel[0].copy()

Next we will extract the instrument response for the chosen waveform from the squirrel(here it is done for data conversion into displacement):

In [ ]:
response = sq.get_response(tr).get_effective(
    input_quantity='displacement')  # Get response to get 'displacement' signal

For quick inspection the response can be printed or even plotted:

In [ ]:
# Printing response information
print(response)

In [ ]:
# Import pyrocko response plotting routine
from pyrocko.plot import response as response_plot

# Plotting the frequency depending response
response_plot.plot(
    responses=[response],
    fmin=0.001,
    fmax=400.,
    dpi=75.);

So, as there is a instrumental response for our chosen trace we can now use it to obtain the ground displacements during the recorded earthquake for the trace:

In [ ]:
tr_rest = tr.transfer(
    0.,  # Rise time of the taper
    transfer_function=response,
    freqlimits=(0.001, 0.01, 10., 20.),
    invert=True,
    demean=True)

Comparing to the old raw trace shows the differences:

In [ ]:
# Assing location code to distinguish waveforms in snuffler plot
tr_rest.set_location('DI')

# Plot the ground displacement
snuffle([tr_rest, tr]);

## Summary <a class="anchor" id="sum"></a>

Here we have covered simple techniques to 
* **load** StationXML into Pyrocko,
* convert StationXML to Pyrocko Stations with **get_pyrocko_stations**
* visualization techniques of station locations and responses,
* simple manipulation steps to add a new station (**Station** and **Channel**),
* writing of stations into a StationXML or Pyrocko YAML file,
* removal of the instrumental response from raw waveforms.